In [1]:
import joblib
import pandas as pd
import keras
from sklearn.preprocessing import StandardScaler
import numpy as np

2025-10-04 12:05:08.950102: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
filename = 'kepler_db.csv'

df = pd.read_csv(filename, comment='#')
cols_to_drop = [
    'rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_vet_stat', 'koi_vet_date',
    'koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 
    'koi_fpflag_ec', 'koi_disp_prov', 'koi_comment', 'koi_eccen', 'koi_eccen_err1', 
    'koi_eccen_err2', 'koi_longp', 'koi_longp_err1', 'koi_longp_err2', 'koi_ingress', 
    'koi_ingress_err1', 'koi_ingress_err2',  'koi_sma_err1', 'koi_sma_err2', 'koi_incl_err1', 
    'koi_incl_err2', 'koi_teq_err1', 'koi_teq_err2', 'koi_limbdark_mod', 'koi_ldm_coeff4', 
    'koi_ldm_coeff3', 'koi_tce_plnt_num', 'koi_tce_delivname', 'koi_quarters', 
    'koi_bin_oedp_sig', 'koi_trans_mod', 'koi_model_dof', 'koi_model_chisq', 
    'koi_datalink_dvr', 'koi_datalink_dvs', 'koi_sage', 'koi_sage_err1', 'koi_sage_err2'
]
df_clean = df.drop(columns=cols_to_drop).reset_index(drop=True)

In [3]:
Y = df_clean['koi_disposition'].map({'FALSE POSITIVE': 0, 'CONFIRMED': 1})
X = df_clean.drop(columns=['koi_disposition'])
X_filled = X.fillna(0)
X_encoded = pd.get_dummies(X_filled, drop_first=False).astype(np.float32)

mask = Y.isna()
X_encoded = X_encoded[mask]

In [4]:
scaler = joblib.load('kepler_scaler.pkl')
X_scaled = scaler.transform(X_encoded).astype(np.float32)

labels = ['FALSE POSITIVE', 'CONFIRMED']
model = keras.models.load_model('kepler.keras')
pred_org = model.predict(X_scaled)
pred = (pred_org >= 0.5).astype(int).flatten()

2025-10-04 12:05:11.960593: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-10-04 12:05:11.960689: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:171] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-10-04 12:05:11.960709: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:176] retrieving CUDA diagnostic information for host: MSI
2025-10-04 12:05:11.960714: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] hostname: MSI
2025-10-04 12:05:11.961199: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] libcuda reported version is: 580.65.6
2025-10-04 12:05:11.961250: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:194] kernel reported version is: 580.65.6
2025-10-04 12:0

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [5]:
candidates_meta = df.loc[mask, ['kepid', 'kepoi_name']]

with open('kepler_predictions.csv', 'w') as f:
    f.write('kepid,kepoi_name,koi_disposition_pred,koi_disposition_pred_value\n')
    for i, (_, row) in enumerate(candidates_meta.iterrows()):
        f.write(f"{row['kepid']},{row['kepoi_name']},{labels[pred[i]]},{pred_org[i][0]}\n")